In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("./data/clean_data.csv")

In [3]:
data.head()

,Unnamed: 0,id,limit_bal,age,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,...,female,education_1,education_2,education_3,education_4,education_5,education_6,marriage_1,marriage_2,marriage_3
0,0,1,20000.0,24,3913.0,3102.0,689.0,0.0,0.0,0.0,...,1,0,1,0,0,0,0,1,0,0
1,1,2,120000.0,26,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,...,1,0,1,0,0,0,0,0,1,0
2,2,3,90000.0,34,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,...,1,0,1,0,0,0,0,0,1,0
3,3,4,50000.0,37,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,...,1,0,1,0,0,0,0,1,0,0
4,4,5,50000.0,57,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,...,0,0,1,0,0,0,0,1,0,0


#### Feature selection

In [4]:
data.columns

Index(['Unnamed: 0', 'id', 'limit_bal', 'age', 'bill_amt1', 'bill_amt2',
       'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6', 'pay_amt1',
       'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6',
       'default_payment_next_month', 'pay_1_-1', 'pay_1_0', 'pay_1_1',
       'pay_1_2', 'pay_1_3', 'pay_1_4', 'pay_1_5', 'pay_1_6', 'pay_1_7',
       'pay_1_8', 'pay_2_-1', 'pay_2_0', 'pay_2_1', 'pay_2_2', 'pay_2_3',
       'pay_2_4', 'pay_2_5', 'pay_2_6', 'pay_2_7', 'pay_2_8', 'pay_3_-1',
       'pay_3_0', 'pay_3_1', 'pay_3_2', 'pay_3_3', 'pay_3_4', 'pay_3_5',
       'pay_3_6', 'pay_3_7', 'pay_3_8', 'pay_4_-1', 'pay_4_0', 'pay_4_1',
       'pay_4_2', 'pay_4_3', 'pay_4_4', 'pay_4_5', 'pay_4_6', 'pay_4_7',
       'pay_4_8', 'pay_5_-1', 'pay_5_0', 'pay_5_2', 'pay_5_3', 'pay_5_4',
       'pay_5_5', 'pay_5_6', 'pay_5_7', 'pay_5_8', 'pay_6_-1', 'pay_6_0',
       'pay_6_2', 'pay_6_3', 'pay_6_4', 'pay_6_5', 'pay_6_6', 'pay_6_7',
       'pay_6_8', 'female', 'education_1', 'education_2'

#### Linear Regression Model

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, mean_squared_error
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.preprocessing import normalize, MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [6]:
# se quitan columas para evitar multicolinealidad
df_X = data.drop(['pay_1_-1', 'pay_1_0', 'pay_1_1',
        'pay_1_2', 'pay_1_3', 'pay_1_4', 'pay_1_5', 'pay_1_6', 'pay_1_7',
        'pay_1_8', 'pay_2_-1', 'pay_2_0', 'pay_2_1', 'pay_2_2', 'pay_2_3',
        'pay_2_4', 'pay_2_5', 'pay_2_6', 'pay_2_7', 'pay_2_8', 'pay_3_-1',
        'pay_3_0', 'pay_3_1', 'pay_3_2', 'pay_3_3', 'pay_3_4', 'pay_3_5',
        'pay_3_6', 'pay_3_7', 'pay_3_8', 'pay_4_-1', 'pay_4_0', 'pay_4_1',
        'pay_4_2', 'pay_4_3', 'pay_4_4', 'pay_4_5', 'pay_4_6', 'pay_4_7',
        'pay_4_8', 'pay_5_-1', 'pay_5_0', 'pay_5_2', 'pay_5_3', 'pay_5_4',
        'pay_5_5', 'pay_5_6', 'pay_5_7', 'pay_5_8', 'pay_6_-1', 'pay_6_0',
        'pay_6_2', 'pay_6_3', 'pay_6_4', 'pay_6_5', 'pay_6_6', 'pay_6_7',
        'pay_6_8', 'education_1', 'education_2', 'education_3', 'age',
        'education_4', 'education_5', 'education_6',
        'marriage_3', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt1', 'limit_bal',
        'id', 'Unnamed: 0'], axis=1)
df_y = data['default_payment_next_month']

vif = pd.Series([variance_inflation_factor(df_X.values, i) for i in range(df_X.shape[1])], index=df_X.columns)
print(vif)

df_X = df_X.drop(['default_payment_next_month'], axis=1)

if 0:
    df_X-=np.mean(df_X) # the -= means can be read as x = x- np.mean(x)
    df_X/=np.std(df_X) # the /= means can be read as x = x/np.std(x)

df_X.head()

bill_amt6                     1.720155
pay_amt1                      1.318214
pay_amt2                      1.240589
pay_amt3                      1.272491
pay_amt4                      1.243965
pay_amt5                      1.261091
pay_amt6                      1.175176
default_payment_next_month    1.296231
female                        2.479606
marriage_1                    2.167310
marriage_2                    2.209689
dtype: float64


,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,female,marriage_1,marriage_2
0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1,1,0
1,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1,0,1
2,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,1,0,1
3,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,1,1,0
4,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,1,0


In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=3)

model1 = LogisticRegression(solver='liblinear', C=10)
model1.fit(X_train, y_train)

y_pred = model1.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print('\nAccuracy Score for model1: ', accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      1.00      0.88      4748
           1       0.74      0.02      0.04      1252

    accuracy                           0.79      6000
   macro avg       0.77      0.51      0.46      6000
weighted avg       0.78      0.79      0.71      6000

[[4738   10]
 [1223   29]]

Accuracy Score for model1:  0.7945


#### SVM Model

In [9]:
output = 'default_payment_next_month'

cols = [ f for f in data.columns if data.dtypes[ f ] != "object"]
cols.remove(output)
cols.remove('id')
cols.remove('Unnamed: 0')

In [10]:
features = cols

In [11]:
features.remove('bill_amt1')
features.remove('bill_amt2')
features.remove('bill_amt3')
features.remove('bill_amt4')
features.remove('bill_amt5')
features.remove('bill_amt6')
features.remove('marriage_3')
features.remove('education_4')
features.remove('education_5')
features.remove('education_6')

##### Data scaling

In [12]:
from sklearn.preprocessing import StandardScaler

X = data[features].values    
y = data[output].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scX = StandardScaler()
X_train = scX.fit_transform(X_train)
X_test = scX.transform(X_test)

##### Model 

In [13]:
from sklearn import svm

svm_classifier = svm.SVC(kernel="rbf")
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Accuracy on Test Set for kernel-SVM = %.2f" % ((cm[0,0] + cm[1,1] ) / len(X_test)))

[[4422  219]
 [ 879  480]]
Accuracy on Test Set for kernel-SVM = 0.82


In [14]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

results = {}
results['accuracy']  = accuracy_score(y_test, y_pred)
results['recall']    = recall_score(y_test, y_pred, average=None)
results['precision'] = precision_score(y_test, y_pred, average=None)

for key, value in results.items():
    print("%s: %s" % (key, value))

accuracy: 0.817
recall: [0.95281189 0.35320088]
precision: [0.83418223 0.68669528]


In [15]:
# minimizando los falsos positivos
svm_classifier = svm.SVC(C=0.1, kernel="rbf", class_weight={0: 1, 1: 2})
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print("Accuracy on Test Set for kernel-SVM = %.2f" % ((cm[0,0] + cm[1,1] ) / len(X_test)))

[[4226  415]
 [ 738  621]]
Accuracy on Test Set for kernel-SVM = 0.81


In [16]:
results['accuracy']  = accuracy_score(y_test, y_pred)
results['recall']    = recall_score(y_test, y_pred, average=None)
results['precision'] = precision_score(y_test, y_pred, average=None)

for key, value in results.items():
    print("%s: %s" % (key, value))

accuracy: 0.8078333333333333
recall: [0.91057962 0.45695364]
precision: [0.85132957 0.59942085]


#### Model 3

In [ ]:
####